In [2]:
import  pymysql



In [14]:
con=pymysql.connect(host='localhost',user='root',password='test',database='Analysis_Data',charset='utf8')
cur=con.cursor()
sql='select * from test'
cur.execute(sql)
all=cur.fetchall()
print(all)
cur.close()
con.close()

((1, '测试'), (1, '测试'))


In [13]:
con=pymysql.connect(host='localhost',user='root',password='test',database='Analysis_Data',charset='utf8')
cur=con.cursor()
isertSql='insert into test (id,name) VALUES (1,"测试")'
cur.execute(isertSql)
con.commit()
cur.close()
con.close()

https://blog.csdn.net/weixin_44786530/article/details/123066819 dataframe数据批量插入db
https://blog.csdn.net/weixin_45914452/article/details/120585861 字段筛选

In [17]:
#使用pandas读取excel
import pandas as pd #导入Pandas
df_fission = pd.read_csv('易速鲜花增长模型.csv') #载入数据


In [19]:
class sqlConfig:
    db_name = "Analysis_Data"     # 数据库名
    db_user = "root"              # 用户名
    db_host = "localhost"         # IP地址
    db_port = 3306                # 端口号
    db_passwd = "test"            # 密码


In [26]:

from sqlalchemy import create_engine

#excel批量写入db
def insert_data_by_pandas(dataframe, table_name, if_exists='append'):
    '''
    通过dataframe 向 sql 中插入表，此方法缺点是若表已存在，不能替换表中部分重复数据，只能替换/追加整张表
    :param dataframe: pd.Dataframe类型
    :param table_name: 插入的表名
    :param if_exists: {'fail', 'replace', 'append'}, default 'fail'
        - fail: If table exists, do nothing.
        - replace: If table exists, drop it, recreate it, and insert data.
        - append: If table exists, insert data. Create if does not exist.
    :return:
    '''

    # 初始化数据库连接，使用pymysql模块
    db_info = {
        'user': sqlConfig.db_user,
        'password': sqlConfig.db_passwd,
        'host': sqlConfig.db_host,
        'port': sqlConfig.db_port,
        'database': sqlConfig.db_name
    }

    engine = create_engine(
        'mysql+pymysql://%(user)s:%(password)s@%(host)s:%(port)d/%(database)s?charset=utf8' % db_info,
        encoding='utf-8'
    )

    dataframe.to_sql(table_name, engine, if_exists=if_exists, index=False, chunksize=100)

In [52]:
insert_data_by_pandas(df_fission[(df_fission['用户码']>200)&(df_fission['用户码']<=300)],'increase_data')

In [46]:
#从db读取数据，转换成dataframe，生成csv文件
import pandas as pd
import pymysql 
 
con = pymysql.connect(host='localhost',user='root',password='test',database='Analysis_Data',charset='utf8') # 连接
cur = con.cursor()
sql='select * from increase_data where `用户码`<=100;'

def read_table(cur, sql): # sql_order is a string
    try:
        cur.execute(sql) # 多少条记录
        data  = cur.fetchall(  )
    
        columnDes = cur.description #获取连接对象的描述信息 
        columnNames = [columnDes[i][0] for i in range(len(columnDes))]    
        frame = pd.DataFrame([list(i) for i in data],columns=columnNames)    
#         frame = pd.DataFrame(list(data),columns=columnNames)
    except: #, e:
        frame = pd.DataFrame()
        # print e
        # continue 
    return frame

df = read_table(cur, sql)
df.to_csv('易速鲜花增长模型100.csv');
 
# con.commit()
cur.close()
con.close()

 